<a href="https://colab.research.google.com/github/rs0927/PyTorch-01/blob/main/PyTorch_training_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv")


In [4]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [5]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)

In [6]:
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=0.2)

In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [9]:
# converting numpy arrays to pytorch tensors
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [10]:
class MyNeuralNetwork():

  def __init__(self,X):

    self.weights = torch.rand(X.shape[1],1,dtype=torch.float64,requires_grad=True)
    self.bias = torch.zeros(1, dtype=torch.float64, requires_grad=True)


  def forward(self, X):
    z = torch.matmul(X, self.weights) + self.bias
    y_pred = torch.sigmoid(z)
    return y_pred

  def loss_function(self, y_pred, y):
    # Clamp predictions to avoid log(0)
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

    # Calculate loss
    loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
    return loss


In [18]:
learning_rate = 0.1
epochs = 1000

In [19]:
# create model
model = MyNeuralNetwork(X_train_tensor)

# define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model.forward(X_train_tensor)

  # loss calculate
  loss = model.loss_function(y_pred, y_train_tensor)

  # backward pass
  loss.backward()

  # parameters update
  with torch.no_grad():
    model.weights -= learning_rate * model.weights.grad
    model.bias -= learning_rate * model.bias.grad

  # zero gradients
  model.weights.grad.zero_()
  model.bias.grad.zero_()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 2.9610743785975977
Epoch: 2, Loss: 2.8149997572211976
Epoch: 3, Loss: 2.6674562264455197
Epoch: 4, Loss: 2.5149828609557896
Epoch: 5, Loss: 2.3633449832123707
Epoch: 6, Loss: 2.207689231328673
Epoch: 7, Loss: 2.054115175887052
Epoch: 8, Loss: 1.8995469785425148
Epoch: 9, Loss: 1.7457536213992133
Epoch: 10, Loss: 1.5979214604622343
Epoch: 11, Loss: 1.4597380824845403
Epoch: 12, Loss: 1.3305775180148895
Epoch: 13, Loss: 1.2157291527717986
Epoch: 14, Loss: 1.1165623688490103
Epoch: 15, Loss: 1.03379825624536
Epoch: 16, Loss: 0.9672073818373969
Epoch: 17, Loss: 0.9154862235982308
Epoch: 18, Loss: 0.8763922141787769
Epoch: 19, Loss: 0.8471635410296068
Epoch: 20, Loss: 0.8251035217947642
Epoch: 21, Loss: 0.808022049006725
Epoch: 22, Loss: 0.7943549099102366
Epoch: 23, Loss: 0.7830639867222776
Epoch: 24, Loss: 0.7734811913329889
Epoch: 25, Loss: 0.7651767538168196
Epoch: 26, Loss: 0.7578682512480044
Epoch: 27, Loss: 0.7513632633203223
Epoch: 28, Loss: 0.7455248219256848
Epoch:

In [20]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.6140350699424744
